<h1 style="color:blue">Praktikum 3. Morfoloogilise analüüsi erijuhud</h1>
<h3 style="color:blue">Korpusepõhine ühestamine, morfoloogiline analüüs kasutajasõnastiku abil ja Giellatekno märgendid</h3>

Tänases praktikumis jätkame morfoloogilise analüüsi ja ühestamise teemadega. Kuna keerukamad analüüsitasemed (süntaks, semantika) toetuvad morfoloogilise analüüsi tulemustele, siis on oluline selle taseme analüüsile rohkem tähelepanu pöörata. Eriti oluliseks muutub see siis, kui analüüsida tuleb tekste, mille keelekasutus erineb kirjakeelest, nt netikeelt, slängi vms eesti keele allkeelt. Samuti võib sõnade morfoloogilist analüüsi parandada see, kui arvestame sõnade kasutust laiemas kontekstis: terves tekstis või _korpuses_ ¹.

   * ¹ _korpus_ = loomuliku keele tekstide kogu. Täpsema määratluse korpuste kohta lingvistikas leiab [siit](https://et.wikipedia.org/wiki/Keelekorpus).

## Teksti- ja korpusepõhine morfoloogiline ühestamine

Vaikimisi EstNLTK's kasutatav morfoloogiline ühestamine, millega tutvusite eelmises praktikumis, toimetab ühe lause piires. Lühidalt kirjeldades: statistilist / masinõppe lähenemist kasutades leitakse igale sõnale kontekstist lähtuvalt kõige tõenäolisem morfoloogiline analüüs, sealjuures üksikule sõnale analüüsi valimisel lähtutakse 1-2 eelneva sõna analüüsidest. Detailsemalt kirjeldab statistilist morfoloogilist ühestamist [see artikkel](http://www.cl.ut.ee/yllitised/kk_yhest_1998.pdf).

Praktikas töötab lausepõhine ühestamine küllaltki hästi, aga selle täpsust on võimalik siiski veelgi suurendada. Ühe lause piires mitmeseks jäävad sõnad võivad mujal tekstis esineda üheselt tõlgendataval kujul, seega, kui kasutada ühestamisel laiemat konteksti kui üks lause, saame lahendada lausepõhise ühestamise poolt lahendamata jäänud või valesti lahendatud mitmesusi.

Enne kui läheme edasi, tuleb ära rääkida üks **tehniline, aga siiski oluline jutt**. Eelmises praktikumis vaatasime mitmeid mugavaid variante teksti morfoloogiliste analüüside kättesaamiseks, nt `text.analysis` andis meile kõigi sõnade analüüside järjendi, `text.lemmas` lemmade järjendi, `text.postags` sõnaliikide järjendi jne. Näiliselt on need `Text` objekti atribuudid, aga tegelikult peituvad nende taga nn _mugavusfunktsioonid_. Väljakutsumisel kontrollivad mugavusfunktsioonid kõigepealt, kas morfoloogiline analüüs on tehtud (kui pole, siis teevad selle), ning seejärel nopivad analüüsist välja nõutud tulemused (kas siis lemmad, sõnaliigid vms). Mugavusfunktsioonide kasutamisel tuleb olla aga ettevaatlik, et mitte komistada järgmiste probleemide otsa:

* mugavusfunktsioonide (`text.words`, `text.analysis`, `text.lemmas` jne) abil **ei ole võimalik** `Text` objekti sisu muuta. Omistamise saab küll teha (nt `text.words=[]`), aga tegelik sisu sellest ei muutu. Kuna `Text` objekt on sõnastiku-tüüpi andmestruktuur, siis saab selle sisu muuta ainult nii, nagu muudetakse sõnastikku, nt omistades `text['words']=[]`;

* mugavusfunktsioonid (`text.words`, `text.analysis`, `text.lemmas` jne) jäädvustavad esimesel väljakutsumisel tagastatava sisu mälupuhvrisse ja edaspidi **tagastavad vana sisu**. Seega, kui muudate `Text` objekti ja kutsute pärast muutmist mugavusfunktsiooni uuesti  välja, ei näe te muutmise tulemust. Ainult siis, kui pöördute `Text` objekti poole kasutades sõnastiku võtmeid (nt `text['words']`), on garanteeritud, et saate kätte kõige värskema sisu;

Arvestades neid probleeme ning seda, et käesolevas praktikumis tuleb üsna palju tegemist morfoloogiliste analüüside kohendamise / muutmisega, siis proovime siin kasutada mugavusfunktsioone nii vähe kui võimalik. Andmetele ligipääs on sel viisil küll natukene keerukam, aga samas on garanteeritud, et kõik toimib nii, nagu peab.

Uurime lause- ja korpusepõhise morfoloogilise ühestamise erinevust järgmise "näitekorpuse" varal:

In [1]:
corpus = ['Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.',\
          'Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.', \
          'Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.']

Sisuliselt käsitleme järjendi `corpus` iga elementi "ühe tekstina". 

Kõigepealt rakendame korpusel morfoloogilist analüüsi koos tavalise, lausepõhise ühestamisega. Kasutame selleks `Text` objekti meetodit `tag_analysis()`. Sisuliselt teostab see meetod morfoloogilise analüüsi vastavalt sellele, millised olid parameetrite `guess`, `disambiguate` ja `propername` väärtused `Text` objekti loomisel (eelmise praktikumi teema) ning kui ühtegi nimetatud parameetritest ei kasutata, eeldab vaikimisi, et kõik on sisse lülitatud. Pärast teksti ühestamist trükime välja sõnad, mille analüüsid jäid mitmeseks. Kasutame sealjuures väikest tsükli-nõksu, et kuvada morfoloogilise analüüsi tulemused nii, et oluline info oleks ühel real. Väljastame iga sõna analüüside osast ainult lemma, sõnaliigi ja käände-/pöördeinfo:

In [2]:
from estnltk import Text

for text_str in corpus:
    text = Text(text_str)
    print(text)
    # Teostame tavalise, lausepõhise teksti ühestamise
    text.tag_analysis()
    # Trükime välja mitmeseks jäänud sõnade analüüsid (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text['words']:
        if len(word['analysis']) > 1:
            print( word['text'],'=>',[(a['lemma'],a['partofspeech'],a['form']) for a in word['analysis']] )

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
kohale => [('koht', 'S', 'sg all'), ('koha', 'S', 'sg all')]
kuigi => [('kuigi', 'D', ''), ('kuigi', 'J', '')]
Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
koha => [('koht', 'S', 'sg g'), ('koha', 'S', 'sg g')]
mail => [('maa', 'S', 'pl ad'), ('mai', 'S', 'sg ad')]
Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.
summaga => [('summ', 'S', 'sg kom'), ('summa', 'S', 'sg kom')]
on => [('olema', 'V', 'b'), ('olema', 'V', 'vad')]


Nagu näha, on ühestamise tulemustes parandamisruumi küll. Proovime korpusepõhist ühestamist.

EstNLTK's klass `Disambiguator` sisaldab meetodit `disambiguate()`, millele antakse sisendiks analüüsitav korpus järjendi kujul. See võibki olla sõnede järjend, nagu meie näitejärjend `corpus`, aga see võib olla ka järjend, mis koosneb `Text` objektidest. Viimasel juhul on oluline meeles pidada, et kui annate sisendiks järjendi `Text` objektidest, millele on juba tehtud morfoloogiline analüüs, siis vanad analüüsitulemused kustutatakse ning analüüsiga alustatakse nullist.

Sisuliselt teostab meetod `disambiguate()` nii tavalise kui ka korpusepõhise ühestamise ning tagastab järjendi `Text` objektidest:

In [3]:
from estnltk import Disambiguator
disamb = Disambiguator()

# Rakendame korpusepõhist ühestamist näitekorpusel
disambiguated_texts = disamb.disambiguate(corpus)

Uurime tulemusi -- väljastame uuesti mitmesed analüüsid:

In [4]:
for text in disambiguated_texts:
    print(text.text)
    # Trükime välja mitmeseks jäänud sõnade analüüsid (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text['words']:
        if len(word['analysis']) > 1:
            print( word['text'],'=>',[(a['lemma'],a['partofspeech'],a['form']) for a in word['analysis']] )

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
kuigi => [('kuigi', 'D', ''), ('kuigi', 'J', '')]
Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.
on => [('olema', 'V', 'b'), ('olema', 'V', 'vad')]


Lisaks on korpusepõhisest ühestamisest abi pärisnimeanalüüside korrastamisel. Kui mingi tavaline sõna on korpuse tekstides tõenäoliselt kasutusel pärisnimena (st esineb suure algustähega ka lausete keskel), siis valib korpusepõhine ühestaja pärisnime-analüüsid selle sõna tavapäraste analüüside asemel. Uurime, millised analüüsid valiti näitekorpuse suure tähega algavatele sõnadele:

In [5]:
for text in disambiguated_texts:
    print(text.text)
    # Trükime välja suure tähega algavate sõnade analüüsid
    for word in text['words']:
        if word['text'].istitle():
            print( word['text'],'=>',[(a['lemma'],a['partofspeech'],a['form']) for a in word['analysis']] )

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
Esimesele => [('esimene', 'O', 'sg all')]
Jänes => [('Jänes', 'H', 'sg n')]
Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Lõpparvestuses => [('lõpparvestus', 'S', 'sg in')]
Konnale => [('Konn', 'H', 'sg all')]
Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
Jänes => [('Jäne', 'H', 'sg in')]
Uus => [('uus', 'A', 'sg n')]
Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.
Konn => [('Konn', 'H', 'sg n')]
Uue => [('uus', 'A', 'sg g')]


Nagu näha, on sõnad _Jänes_ ja _Konn_ saanud pärisnime sõnaliigimärgendi (`'H'`).

**_Tekstipõhine ühestamine_**. Korpusepõhise ühestaja abil võib teostada ka tekstipõhist ühestamist ehk siis: ühestamist, kus "korpuseks" on üks tekst. 
See võib olla vajalik siis, kui korpuse tekstid on väga heterogeensed ning üks sõna võib esineda kahes tekstis täiesti erinevates tähendustes (omada erinevat lemmat). 
Sellisel juhul võib liiga lai ühestamiskontekst (terve korpus) ajada lemmad valeks ning parem on piirduda väiksema kontekstiga.

Tekstipõhiseks ühestamiseks tuleb lihtsalt ühestamismeetodile anda ette järjend, mis sisaldab ühte teksti:

       disamb = Disambiguator()
       disambiguated_texts = disamb.disambiguate([my_text])
       
ning pärast tulemusteks saadud järjendist tekst uuesti välja noppida:
       
       disambiguated_text = disambiguated_texts[0]

Korpusepõhise ühestamise teooriat kirjeldab [see artikkel](https://www.etis.ee/File/DownloadPublic/ce4606c7-41ba-4059-ac6d-6db2e40442e9?name=Fail_FAIA247-0082.pdf&type=application%2Fpdf). EstNLTK korpusepõhise ühestaja liidest kirjeldab detailsemalt [see abimaterjal](https://estnltk.github.io/estnltk/1.4.1/tutorials/disambiguation.html).

### Ülesanne 1: Kui palju vähendab korpusepõhine ühestamine mitmesust? (1,25p)

Esimeseks ülesandeks on uurida, kui palju vähendab **korpusepõhine ühestamine** mitmesust. Kataloogis `'aja_sloleht_1999_04_k'` on SL Õhtulehe ajaleheartiklid. 
Igas failis on üks artikkel. 
Lugege sealt tekstid sisse ning teostage morfoloogiline ühestamine kahel erineval viisil: 1) tavaline, nn lausepõhine ühestamine, 2) korpusepõhine ühestamine, kus korpuseks on kõik artiklid, st terve kataloogi sisu.
Leidke ja väljastage mõlema ühestamisetapi kohta mitmeseks jäänud sõnade arv ja osakaal kõigist sõnadest (%).

Lisajuhis:

 * Failide kõvakettalt sisselugemine on üldiselt ressurssinõudlik operatsioon; seetõttu on soovitav ülesanne lahendada nii, et loete kõigi failide sisud kõigepealt mällu (nt järjendisse) ja lahendate alamülesanded 1) ja 2) mälus olevatel andmetel;

### Ülesanne 2: Pärisnimede ühestamise kvaliteedi uurimine (1,75 p)

Korpuste kasutajaid huvitab sageli pärisnimede otsimine. 
Järgnev ülesanne on uurida, kas ja kuidas aitab **tekstipõhine ühestamine** kaasa pärisnimeanalüüside kvaliteedi paranemisele.
Looge programm, mis analüüsib korpust kataloogis `'aja_sloleht_1999_04_k'` ning väljastab kõik laused, kus leidub sõnu, mille puhul tavaline ühestamine ei andnud üldse pärisnime analüüsi, aga tekstipõhise ühestamise järel sai sõna ühese pärisnime analüüsi. Programm väljastab lausete tekstid, ühese pärisnimeanalüüsi saanud sõnad ja nende analüüside muutuse. Näide:

    >> aja_sloleht_1999_04_15__20.txt
    Matkamiksi peakorraldaja Andrus Nõmm tunnistas , et turvafirma Desperada kuuele töötajale ei ole tal midagi ette heita .
     Nõmm
       [('nõmm', 'S', 'sg n')]
       ==>
       [('Nõmm', 'H', 'sg n')]

    “ Olin ka ise seal , ” rääkis Nõmm .
     Nõmm
       [('nõmm', 'S', 'sg n')]
       ==>
       [('Nõmm', 'H', 'sg n')]

    Viha noorukite peale Kare ei pea , sest kaupluse alkoholikäive suurened messi ajal mitu korda .
     Kare
       [('kare', 'A', 'sg n')]
       ==>
       [('Kare', 'H', 'sg n')]
    
Kuidas sellele ülesandele võiks läheneda? Teostage kõigepealt kataloogi `'aja_sloleht_1999_04_k'` failidel tekstipõhised ühestamised ning paigutage ühestamise tulemused (morfoloogiliselt ühesed `Text` objektid) järjendisse.
Seejärel läbige tekstid lause-kaupa ning leidke igast lausest sõnad, mis said (tekstipõhise) ühestamise tulemusena ühese pärisnime analüüsi (sõnaliik: `'H'`).
Kui lauses leidus vähemalt üks selline sõna, teostage kogu lausel (lause tekstil) morfoloogiline analüüs uuesti, aga kasutage nüüd tavalist ühestamist (eelmise ülesande ühestamine 1).
Lõpuks võrrelge kahe ühestamise tulemusi pärisnimeanalüüsi saanud sõnade peal ja leidke sobiva muutusega sõnad.
Väljastage laused, kuhu tekkisid ühese pärisnimeanalüüsiga sõnad, ning vastavad sõnad ja nende analüüside muutused (väljund võiks olla eeltoodud näitega sarnane).

Veel näpunäiteid:

  * Kui teete tsükli, mis käib üle korpuse kõigi tekstide, arvestage sellega, et korpus on üksjagu mahukas. Alguses on katsetamiseks soovitav teha kõigepealt tsükkel, mis katkestatakse [`break`](https://docs.python.org/3.5/tutorial/controlflow.html#break-and-continue-statements-and-else-clauses-on-loops) käsuga, kui teatud arv esimesi tekste (või lauseid) on juba läbitud. Kui olete jõudnud niikaugele, et programm töötab ilusti väiksel osal korpusest, siis saab ka katkestusest loobuda ja panna tsükli(d) jooksma üle kogu korpuse;

## Morfoloogiline analüüs ja ühestamine kasutajasõnastiku abil

Morfoloogiline analüsaator töötab kõige paremini tekstidel, mis järgivad kirjakeele norme. Kuigi analüsaator sisaldab ka tundmatute sõnade oletajat, ei saa selle täpsusele lootma jääda, kui analüüsitavad tekstid erinevad kirjakeelest suurel määral. Seetõttu kasutatakse netikeele vms eesti keele allkeele automaatseks analüüsimiseks nn _kasutajasõnastiku lähenemist_. Kõigepealt luuakse (poolautomaatselt) kasutajasõnastik, kus on toodud kirjakeelest erinevate sõnavormide korrektsed morfoloogilised analüüsid. Seejärel teostatakse automaatne morfoloogiline analüüs ilma ühestamiseta ning analüüsi tulemustes asendatakse kasutajasõnastikus olevate sõnade (automaat)analüüsid nende korrektsete analüüsidega. Viimase sammuna rakendatakse morfoloogiliselt analüüsitud tekstil ühestamist.

Vaatame seda protsessi samm-sammult ühe netikeele lause näitel.

In [6]:
text = "Sa ajad sássi inimmeste erinevad käsitlusviisid ja lóodusnähhtuste kinndla vahekorra."

Meie näide on lihtne selles mõttes, et kuigi siin on mitte-kirjakeelseid sõnavorme (nt _inimmeste_, _lóodusnähhtuste_), leidub kõigile sõnadele siiski kirjakeeles vaste. Seega saame tekitada mitte-kirjakeelsete vormide morfoloogilised analüüsid, kui analüüsime vastavaid kirjakeele sõnavorme:

In [7]:
from estnltk import Text

# Loome uue kasutajasõnastiku
user_dict = {}
# Lisame tundmatute sõnade vormid koos neile vastavate kirjakeele sõnade analüüsidega
user_dict["inimmeste"]       = Text('inimeste', guess=False, disambiguate=False, propername=False ).analysis[0]
user_dict["lóodusnähhtuste"] = Text('loodusnähtuste', guess=False, disambiguate=False, propername=False ).analysis[0]
user_dict["kinndla"]         = Text('kindla', guess=False, disambiguate=False, propername=False ).analysis[0]
user_dict["sássi"]           = Text('sassi', guess=False, disambiguate=False, propername=False ).analysis[0]

Tulemuseks saame sellise sõnastiku:

In [8]:
user_dict

{'inimmeste': [{'clitic': '',
   'ending': 'te',
   'form': 'pl g',
   'lemma': 'inimene',
   'partofspeech': 'S',
   'root': 'inimene',
   'root_tokens': ['inimene']}],
 'kinndla': [{'clitic': '',
   'ending': '0',
   'form': 'sg g',
   'lemma': 'kindel',
   'partofspeech': 'A',
   'root': 'kindel',
   'root_tokens': ['kindel']}],
 'lóodusnähhtuste': [{'clitic': '',
   'ending': 'te',
   'form': 'pl g',
   'lemma': 'loodusnähtus',
   'partofspeech': 'S',
   'root': 'loodus_nähtus',
   'root_tokens': ['loodus', 'nähtus']}],
 'sássi': [{'clitic': '',
   'ending': '0',
   'form': '',
   'lemma': 'sassi',
   'partofspeech': 'D',
   'root': 'sassi',
   'root_tokens': ['sassi']},
  {'clitic': '',
   'ending': '0',
   'form': 'adt',
   'lemma': 'sasi',
   'partofspeech': 'S',
   'root': 'sasi',
   'root_tokens': ['sasi']}]}

  * _Aga kui morfoloogiline analüsaator ei oskagi kirjakeelset vastet analüüsida (tegemist on uudissõnaga vms)?_ Siis on põhimõtteliselt võimalik sõna morfoloogiliste analüüside järjend ka ise käsitsi tekitada. Sellisel juhul tuleks aga väga täpselt jälgida, et analüüse kirjeldavad sõnastikud sisaldaksid kõiki nõutud võtmeid (`'clitic'`, `'ending'`, `'form'`, `'lemma'`, `'partofspeech'`, `'root'`, `'root_tokens'`) ja võtmete väärtused oleksid sellises formaadis, mida kasutab morfoloogiline analüsaator.

Järgmise sammuna teostame tekstil morfoloogilise analüüsi ilma ühestamise ja pärisnimede oletamiseta. Oletamise enda jätame sisse (selleks, et lauselõpu punktile ikkagi analüüs külge tuleks).

In [9]:
# Teostame morf analüüsi (oletamisega, aga ilma ühestamiseta)
morph_analysed = Text(text, guess=True, disambiguate=False, propername=False)
morph_analysed = morph_analysed.tag_analysis()

Kasutame taaskord "kompaktset" tulemuste väljastamise viisi:

In [10]:
for word in morph_analysed['words']:
    analysis = [ (a['lemma'], a['partofspeech'], a['form']) for a in word['analysis'] ]
    print( word['text'],'=>',analysis )

Sa => [('sina', 'P', 'sg n')]
ajad => [('aeg', 'S', 'pl n'), ('ajama', 'V', 'd')]
sássi => [('sáss', 'S', 'adt'), ('sáss', 'S', 'sg g'), ('sáss', 'S', 'sg p'), ('sássi', 'S', 'sg g'), ('sássi', 'S', 'sg n')]
inimmeste => [('inimmest', 'S', 'pl p')]
erinevad => [('erinema', 'V', 'vad'), ('erinev', 'A', 'pl n')]
käsitlusviisid => [('käsitlusviis', 'S', 'pl n')]
ja => [('ja', 'J', '')]
lóodusnähhtuste => [('lóodusnähhtune', 'A', 'pl g'), ('lóodusnähhtus', 'S', 'pl g'), ('lóodusnähhtuste', 'S', 'sg g'), ('lóodusnähhtuste', 'S', 'sg n')]
kinndla => [('kinndla', 'S', 'sg g'), ('kinndla', 'S', 'sg n')]
vahekorra => [('vahekord', 'S', 'sg g')]
. => [('.', 'Z', '')]


Nagu tulemustest võib näha, on oletaja segaduses ja pakub üksjagu veidraid variante. Sõnale _kinddla_ ei suudagi oletaja õige sõnaliigiga vastet (ehk siis: omadussõna) pakkuda.

Õnneks on meil kasutajasõnastik, mille abil saame tulemusi korrigeerida. Selleks teeme tsükli üle morfoloogilisi analüüse sisaldava `'words'` kihi ning kirjutame üle kõikide kasutajasõnastikus olevate sõnade analüüsid. On oluline üle rõhutada: selleks, et muutused läbi läheksid, tuleb `Text` objekti poole pöörduda kui sõnastiku poole -- ehk siis võtmete kaudu:

In [11]:
for word in morph_analysed['words']:
    # Kas sõna on kasutajasõnastikus? 
    # Kui on, saame selle analüüsi sealt:
    if word['text'] in user_dict:
        # Kirjutame analüüsi üle kasutajasõnastiku analüüsiga
        word['analysis'] = user_dict[ word['text'] ]
        # Hea on panna ka sõnale märge juurde, et see pärineb
        # kasutajasõnastikust (juhuks, kui hiljem on vaja 
        # eristada tavalisi kirjakeele sõnu kasutajasõnastiku
        # sõnadest)
        word['from_user_dict'] = True

In [12]:
# Veendume, et muutused läksid läbi -- kuvame teksti / sõnastiku sisu:
morph_analysed

{'paragraphs': [{'end': 85, 'start': 0}],
 'sentences': [{'end': 85, 'start': 0}],
 'text': 'Sa ajad sássi inimmeste erinevad käsitlusviisid ja lóodusnähhtuste kinndla vahekorra.',
 'words': [{'analysis': [{'clitic': '',
     'ending': '0',
     'form': 'sg n',
     'lemma': 'sina',
     'partofspeech': 'P',
     'root': 'sina',
     'root_tokens': ['sina']}],
   'end': 2,
   'start': 0,
   'text': 'Sa'},
  {'analysis': [{'clitic': '',
     'ending': 'd',
     'form': 'pl n',
     'lemma': 'aeg',
     'partofspeech': 'S',
     'root': 'aeg',
     'root_tokens': ['aeg']},
    {'clitic': '',
     'ending': 'd',
     'form': 'd',
     'lemma': 'ajama',
     'partofspeech': 'V',
     'root': 'aja',
     'root_tokens': ['aja']}],
   'end': 7,
   'start': 3,
   'text': 'ajad'},
  {'analysis': [{'clitic': '',
     'ending': '0',
     'form': '',
     'lemma': 'sassi',
     'partofspeech': 'D',
     'root': 'sassi',
     'root_tokens': ['sassi']},
    {'clitic': '',
     'ending': '0',
     'f

In [13]:
# Tulemused kompaktsel kujul:
for word in morph_analysed['words']:
    print(word['text'],'=>',[(a['lemma'],a['partofspeech'],a['form']) for a in word['analysis']])

Sa => [('sina', 'P', 'sg n')]
ajad => [('aeg', 'S', 'pl n'), ('ajama', 'V', 'd')]
sássi => [('sassi', 'D', ''), ('sasi', 'S', 'adt')]
inimmeste => [('inimene', 'S', 'pl g')]
erinevad => [('erinema', 'V', 'vad'), ('erinev', 'A', 'pl n')]
käsitlusviisid => [('käsitlusviis', 'S', 'pl n')]
ja => [('ja', 'J', '')]
lóodusnähhtuste => [('loodusnähtus', 'S', 'pl g')]
kinndla => [('kindel', 'A', 'sg g')]
vahekorra => [('vahekord', 'S', 'sg g')]
. => [('.', 'Z', '')]


Viimase sammuna rakendame _lausepõhist / statistilist ühestamist_. Põhimõtteliselt on "sügaval EstNLTK kõhus" olemas eraldi meetod, mis tegelebki ainult lausepõhise ühestamisega. Meetod `estnltk.vabamorf.morf.disambiguate()` saab sisendiks ühe lause sisu (s.o järjendi, mis koosneb lause sõnu kirjeldavatest sõnastikest, iga sõna morf analüüsid võtme `'analysis'` all), ning tagastab "ühestatud koopia" sellest järjendist. Kuna sisend-järjendit ennast ühestamise käigus ei muudeta, siis peame muutuste sisseviimiseks tagastatud järjendist analüüsid välja noppima ning kirjutama algsed analüüsid uutega üle. Kõik kokku käib nii:

In [14]:
from estnltk.vabamorf.morf import disambiguate

sentences = morph_analysed.divide('words', 'sentences')
for sentence in sentences:
    # 1) rakendame lause sõnade järjendil morf ühestamist
    disambiguated = disambiguate(sentence)
    # 2) viime muutused sisse:
    #    asendame algsed analüüsid uute, ühestatud analüüsidega
    for orig, new in zip(sentence, disambiguated):
        orig['analysis'] = new['analysis']

 * **NB!** Eelmises praktikumis kasutasite meetodit `split_by()` teksti tükeldamiseks, nüüd aga kasutame meetodit `divide()`. Kuigi need meetodid teevad näiliselt sama asja, on antud juhul nende erinevus väga oluline. `split_by()` tekitab teksti jagamisel uued `Text` objektid (mis sisaldavad koopiaid vana objekti mingitest alamosadest, nt lausetest), samas `divide()` tagastab meile konkreetsed alamosad vanast objektist (nt laused). Seega, kui rakendaksime ühestamist `split_by()` tulemustel, toimuks ühestamine küll koopia-objektidel, aga vana / terviklik tekst jääks ikkagi muutmata / ühestamata. Meetod `divide()` võimaldab aga muuta vana objekti. Rohkem saab lugeda `divide()` kohta [siit](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#dividing-elements-by-layers) ja `split_by()` kohta [siit](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#splitting-by-layers).

In [15]:
# Tulemused kompaktsel kujul:
for word in morph_analysed['words']:
    print(word['text'],'=>',[(a['lemma'],a['partofspeech'],a['form']) for a in word['analysis']])

Sa => [('sina', 'P', 'sg n')]
ajad => [('aeg', 'S', 'pl n')]
sássi => [('sassi', 'D', '')]
inimmeste => [('inimene', 'S', 'pl g')]
erinevad => [('erinev', 'A', 'pl n')]
käsitlusviisid => [('käsitlusviis', 'S', 'pl n')]
ja => [('ja', 'J', '')]
lóodusnähhtuste => [('loodusnähtus', 'S', 'pl g')]
kinndla => [('kindel', 'A', 'sg g')]
vahekorra => [('vahekord', 'S', 'sg g')]
. => [('.', 'Z', '')]


## Kasutajasõnastiku (pool-)automaatne loomine

Suure korpuse korral on kasutajasõnastiku loomine siiski väga töömahukas tegevus ning täieliku käsitööna mitte eriti mõeldav. Parem lähenemine on kirjakeelsete vastete leidmine vähemalt osaliselt automatiseerida. Järgnevates ülesannetes uurimegi üht sellise automatiseerimisprotsessi alamosa: tundmatutele sõnadele kirjakeelsete vastete leidmist. Kõigepealt peame aga kindlaks tegema, millised sõnad saab üldse tundmatuteks lugeda (arvestades uuritava korpuse sõnestuse eripärasid).

### Ülesanne 3: Tundmatud sõnad (1,5 p)

Looge programm, mis loeb failist `'netikeele_laused.txt'` laused (failis on iga lause eraldi real) ning teostab lausetel morf analüüsi (ilma oletamise, pärisnimede pakkumise ja ühestamiseta). 
Programm leiab, mitu % kõigist sõnadest jäid morf analüsaatorile tundmatuks (st sõnadele ei leitud analüüse), ning leiab 10 lauset, mis sisaldavad kõige rohkem tundmatuid sõnu. 
Leitud 10 lauset ja nendes tundmatuks jäänud sõnad väljastatakse ekraanile (sorteerituna "tundmatute arvu järgi" kahanevalt) ning ühtlasi salvestatakse ka kuhugi andmestruktuuri -- neid lauseid läheb tarvis ka järgmises ülesandes.

  * Sõnade loendamisel tuleks välja jätta 1-tähelised "sõnad", numbritest ja punktuatsioonist koosnevad sõnad;
  * "Tundmatute sõnade" hulka ei tuleks lugeda punktuatsiooni;
  * 10 kõige rohkem tundmatuid sõnu sisaldava lause hulka võib sattuda ka lauseid, milles on sama arv tundmatuid sõnu. Kui viimas(t)ele positsiooni(de)le on valida mitme sama arvu tundmatuid sisaldava lause vahel, siis võib valida lause(d), mis on algses (failis olevas) järjestuses eespool;

### Teisenduskauguse abil vastete leidmine

Nüüd on meil teada, millist laadi sõnadele tuleb hakata kirjakeelseid vasteid otsima. 
Aga kuidas vasteid leida?
Tehniliselt kõige lihtsam lähenemine on kasutada juba valmislahendust -- proovida eelmises praktikumis tutvustatud automaatset [õigekirjakorrektorit](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#correcting-spelling), mida võib ka rakendada tundmatutele sõnadele vastete leidmiseks.
Siiski ei pruugi õigekirjakorrektor netikeele kõige konarlikemates lausetes, kus esineb eri tüüpi vigu ning nii mõnigi kord on tegemist süstemaatilise "teisiti kirjutamisega", kõige paremaid tulemusi anda. 

Üldine meetod, mida rakendab (küll teatavate lisanüanssidega) ka õigekirjakorrektor, on tundmatute sõnade võrdlemine teadaolevate sõnade leksikoniga ning sõnade _sarnasuse_ põhjal vastete väljapakkumine. 
Seega, kui meil on olemas piisavalt mahukas kirjakeele korpus, võime selle põhjal ka ise teha teadaolevate sõnade loendi, noppida loendist välja tundmatute sõnadega _sarnased_ sõnad ja pakkuda neid vasteteks.

_Kuidas hinnata sõnade sarnasust?_ Siin tuleb appi teisenduskaugus (ingl _edit distance_). Sisuliselt mõõdab teisenduskaugus, mitu teisendust / muutust tuleb teha, et saada ühest sõnest teine. Tavaliselt lubatakse kolme tüüpi muutuseid: 1) ühe tähe kustutamine, 2) ühe tähe lisamine, 3) ühe tähe asendamine mingi teise tähega. Teisenduskaugus ütleb, mitu sellist muutust tuleb minimaalselt teha -- mida väiksem arv, seda sarnasemad kaks sõna on.

Teisenduskauguse leidmist võib katsetada funktsiooni `nltk.metrics.distance.edit_distance` abil (funktsiooni sisaldav teek `nltk` peaks teil olema juba installitud, kuna see on üks `estnltk` eelduseid):

In [16]:
from nltk.metrics.distance import edit_distance

edit_distance('pizza', 'pitsa')

2

In [17]:
edit_distance('mõttetu', 'mõtetu')

1

In [18]:
edit_distance('uit', 'uit')

0

Vt ka `nltk` teisenduskauguse funktsiooni [detailsemat kirjeldust](http://www.nltk.org/api/nltk.metrics.html#nltk.metrics.distance.edit_distance).

### Ülesanne 4: Tundmatutele sõnadele vastete leidmine (2 p)

See ülesanne koosneb kahest osast:

#### Ülesanne 4.A: Minu soovitaja (1,25 p)

Looge funktsioon, mis saab sisendiks tundmatu sõna ning tagastab sellele kõige sarnasemad kirjakeele vasted. 
Funktsioon võiks osata pakkuda natukene rohkem vasteid kui EstNLTK [õigekirjakontrollija](https://estnltk.github.io/estnltk/1.4.1/tutorials/text.html#correcting-spelling). 
Seega, esimese sammuna võibki funktsioon kasutada õigekirjakorrektorit vastete leidmiseks, aga kui korrektor midagi pakkuda ei oska, tuleks teisenduskauguse abil ise vasteid otsida. 
Funktsioon tagastab sarnaste sõnade järjendi (ja tühijärjendi, kui vasteid üldse ei leitud).

Teisenduskauguse abil võiks vasteid otsida ajakirjandustekstide sõnavarast (kataloogi `'aja_sloleht_1999_04_k'` tekstidest). 
Kui suures osas ajakirjendustekstide sõnavara tuleks arvestada (mida jätta ja mida võtta), jääb teie otsustada.
Samuti jääb teie otsustada, kui palju ja millisel teisenduskaugusel vasteid tagastatakse (üldine põhimõte on, et mida vähem ja täpsemaid vasteid pakutakse, seda parem).

  * **NB!** Kataloogi 'aja_sloleht_1999_04_k' tekstide sisselugemine ja nende põhjal sõnaloendi moodustamine peaks toimuma ainult üks kord -- kindlasti ei tohiks seda teha igal soovitaja-funktsiooni väljakutsel. 


#### Ülesanne 4.B: Soovitaja rakendamine tundmatute tuvastamiseks (0,75 p)

Rakendage oma soovitaja-funktsiooni ülesandes 3 leitud kümne "kõige konarlikuma" lause tundmatutel sõnadel. Väljastage:

   * laused ning iga lause all nii tundmatud sõnad kui ka neile pakutavad vasted;
   * (üle kõigi lausete) mitmele %-le tundmatutest sõnadest pakutakse vasteid;
   * (üle kõigi lausete) milline on keskmine soovituste arv (ehk siis: mitu vastet keskmiselt iga tundmatu sõna kohta soovitatakse);

Vajadusel täiendage oma funktsiooni nii, et vähemalt 90% tundmatutest sõnadest saaksid soovitusi.

### Soovitustest kasutajasõnastikuni

Eelmises ülesandes arendasime ja katsetasime soovitusfunktsiooni 10 lause tundmatutel sõnadel. 
Praktikas on tavaliselt eesmärgiks terve korpuse analüüs, seega on ka ülesanne üksjagu töömahukam. 
Kuidas praktikas kohandada morfoloogiline analüsaator netikeele tekstide analüüsimiseks, on kirjeldatud detailsemalt [selles artiklis](https://www.etis.ee/File/DownloadPublic/ef492abe-ba06-4e86-9fee-a7e5cd44eeb6?name=Fail_ERYa7.07_Muischnek_pp111-127.pdf&type=application%2Fpdf). Üldine põhimõte on järgmine. Kirjakeelest erinevad ja haruldased sõnad peaksid olema kirjakeelsetest vormidest tuletatavad suhteliselt regulaarsete teisenduste abil. Ehk siis leitavad teisenduskauguse või veelgi spetsiifilisemate teisenduste kaudu; nende lisamine kasutajasõnastikku võiks toimuda suhteliselt automaatselt. Samas, kui sõnavorm pole tuletavav kirjakeelsest sõnast regulaarse teisenduse abil, peaks ta olema sageli kasutatav ("et tema tähendus ja funktsioon kasutajatel meeles püsiks") ja seega korpuse sõnasageduste uurimisel silma torkama. Sellised sõnavormid tuleks siis korpuse sagedusloendite uurimisel välja selgitada ja lisada kasutajasõnastikku käsitsi.

## Giellatekno (GT) morfoloogilised kategooriad

Siiani oleme vaadanud, kuidas parandada oma teksti morfoloogilist analüüsi nii, et see etteantud morfoloogiliste kategooriate süsteemis võimalikult korrektselt analüüsitud saaks. Võib aga juhtuda ka nii, et tekst on küll (piisavalt) korrektselt analüüsitud vaikimisi kasutatavate kategooriate järgi, ent see siiski millegipärast ei rahulda teksti töötlejat. Seetõttu on EstNLTK-s lisaks vaikimisi kasutatavale Filosofti süsteemile ka teine morfoloogiliste kategooriate süsteem: Giellatekno (GT).

**Tehniline vahemärkus**: programm, mis konverteerib kategooriad Filosofti süsteemist Giellatekno süsteemi, kasutab `java`-t. Seega tuleb enne konverteri kasutamist:

  * Installida süsteemi [_Java SE Runtime Environment_](https://www.java.com/en/download/) (versioon >= 1.8);
  
  * Panna `java` käsk süsteemi keskkonnamuutujasse PATH. Windows-i ja Mac-i puhul tehakse seda tüüpiliselt juba installi käigus, aga kui on siiski tarvis seda käsitsi teha, siis detailsemat abi saab [siit](https://java.com/en/download/help/path.xml);

Kuidas kontrollida, kas `java` on juba olemas või kas installimine õnnestus? Käsureakäsk `java -version` peaks kuvama infot installitud `java` versiooni kohta, näiteks midagi taolist:

    java version "1.8.0_171"
    Java(TM) SE Runtime Environment (build 1.8.0_171-b11)
    Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)


Giellatekno morfoloogiliste kategooriate süsteemi saab EstNLTK-s kasutada järgnevalt:

In [19]:
from estnltk.converters.gt_conversion import convert_to_gt
from estnltk import Text

In [20]:
text = Text('Mis siis, et kuuldavasti teevadki Poola torumehed nii Pariisis kui ka Brüsselis torud reaalselt korda.')

In [21]:
# Märgendame tekstile peale Filosofti analüüsi
text.tag_analysis()

{'paragraphs': [{'end': 102, 'start': 0}],
 'sentences': [{'end': 102, 'start': 0}],
 'text': 'Mis siis, et kuuldavasti teevadki Poola torumehed nii Pariisis kui ka Brüsselis torud reaalselt korda.',
 'words': [{'analysis': [{'clitic': '',
     'ending': '0',
     'form': 'pl n',
     'lemma': 'mis',
     'partofspeech': 'P',
     'root': 'mis',
     'root_tokens': ['mis']},
    {'clitic': '',
     'ending': '0',
     'form': 'sg n',
     'lemma': 'mis',
     'partofspeech': 'P',
     'root': 'mis',
     'root_tokens': ['mis']}],
   'end': 3,
   'start': 0,
   'text': 'Mis'},
  {'analysis': [{'clitic': '',
     'ending': '0',
     'form': '',
     'lemma': 'siis',
     'partofspeech': 'D',
     'root': 'siis',
     'root_tokens': ['siis']}],
   'end': 8,
   'start': 4,
   'text': 'siis'},
  {'analysis': [{'clitic': '',
     'ending': '',
     'form': '',
     'lemma': ',',
     'partofspeech': 'Z',
     'root': ',',
     'root_tokens': [',']}],
   'end': 9,
   'start': 8,
   'text': ',

In [22]:
# Teisendame morfoloogilise analüüsi GT formaati - selleks luuakse uus kiht nimega 'gt_words
text = convert_to_gt(text)

In [23]:
text['gt_words']

[{'analysis': [{'clitic': '',
    'ending': '0',
    'form': 'Pl Nom',
    'lemma': 'mis',
    'partofspeech': 'P',
    'root': 'mis',
    'root_tokens': ['mis']},
   {'clitic': '',
    'ending': '0',
    'form': 'Sg Nom',
    'lemma': 'mis',
    'partofspeech': 'P',
    'root': 'mis',
    'root_tokens': ['mis']}],
  'end': 3,
  'start': 0,
  'text': 'Mis'},
 {'analysis': [{'clitic': '',
    'ending': '0',
    'form': '',
    'lemma': 'siis',
    'partofspeech': 'D',
    'root': 'siis',
    'root_tokens': ['siis']}],
  'end': 8,
  'start': 4,
  'text': 'siis'},
 {'analysis': [{'clitic': '',
    'ending': '',
    'form': '',
    'lemma': ',',
    'partofspeech': 'Z',
    'root': ',',
    'root_tokens': [',']}],
  'end': 9,
  'start': 8,
  'text': ','},
 {'analysis': [{'clitic': '',
    'ending': '0',
    'form': '',
    'lemma': 'et',
    'partofspeech': 'J',
    'root': 'et',
    'root_tokens': ['et']}],
  'end': 12,
  'start': 10,
  'text': 'et'},
 {'analysis': [{'clitic': '',
    'en

### Boonusülesanne (0,5p)

Tutvuge giellatekno formaadis morfoloogilise analüüsiga. Selleks teisendage järgneva teksti analüüs GT kujule ning printige välja iga tekstisõna koos selle mõlemas formaadis morfoloogilise analüüsiga (*form*).

*Kaks telgipäeva, 12. ja 13. juunil Tallinnas Vabaduse väljakul läksid väga edukalt. Saime kogeda nii vihma kui ka päikest, mürtsus puhkpillimuusika, ajakirjanikud sagisid ringi ning mis peamine – telgid olid tulvil heategudest. Kas ka sina käisid? Kui ei, tule järgmine aasta, paremat võimalust ei tule.*

Vastake küsimustele:
* Mis on suurim sisuline erinevus GT ja Filosofti analüüside vahel? Vihje: vaadake verbianalüüse
* Kui lähtuda teksti morfoloogilise analüüsi täpsusest ja saagisest, siis millisel juhul oleks mõistlik kasutada GT kuju, millisel juhul Filosofti oma?

Soovi korral rohkem infot verbi morfoloogiliste kategooriate esitamise võimaluste kohta [siit](http://kjk.eki.ee/download_pdf/582)